#텍스트 데이터의 이상한 패턴이 실제 위험요인과 관련 있는지 검정한다 (가설 검정)

1.SBERT로 텍스트 임베딩 생성

2.검정할 대상(위험요인 정하기) : 텍스트 패턴이 실제 위험요인과 통계적으로 관련 있는지를 검정

-> ① 가격/② 건물 유형(building_usage)/ ③ 이미지 이상치(image_anomaly_score)

3.텍스트 패턴이 위험요인과 관련 있는지 검정

4.가설2가 유의하다(p<0.05)이면 이상치 스코어 생성


#1단계: SBERT로 텍스트 임베딩 생성

In [2]:
!pip install -q sentence-transformers tqdm

In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
csv_path = "/content/drive/MyDrive/daangn_list_detail_with_image_score.csv"
df = pd.read_csv(csv_path)

print(df.shape)
print(df.columns)

(1059, 17)
Index(['area', 'identifier', 'description', 'image_count', 'image',
       'building_name', 'building_usage', 'exclusive_area', 'floor',
       'direction', 'maintenance_fee', 'built_year', 'total_floor', 'price',
       'address', 'register_date', 'image_anomaly_score'],
      dtype='object')


In [5]:
assert "description" in df.columns, "description column is missing"

In [7]:
#description 전처리 (필수)
df["description"] = df["description"].fillna("").astype(str).str.strip()

In [8]:
#SBERT 모델 로드
model_name = "jhgan/ko-sbert-multitask"
model = SentenceTransformer(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
#description → SBERT 임베딩 변환
texts = df["description"].tolist()

embeddings = model.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True
)

print("Embedding shape:", embeddings.shape)

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

Embedding shape: (1059, 768)


In [10]:
#DF에 SBERT 벡터 저장 (선택)
# 임베딩을 df에 붙일 필요는 없지만 원한다면:
for i in range(embeddings.shape[1]):
    df[f"sbert_{i}"] = embeddings[:, i]

/tmp/ipython-input-3712171738.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"sbert_{i}"] = embeddings[:, i]
/tmp/ipython-input-3712171738.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"sbert_{i}"] = embeddings[:, i]
/tmp/ipython-input-3712171738.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe =

In [11]:
#임베딩 파일 저장해서 이후 가설검정에서 사용
out_path = "/content/drive/MyDrive/text_embeddings.npy"
np.save(out_path, embeddings)
print("임베딩 저장 완료:", out_path)

임베딩 저장 완료: /content/drive/MyDrive/text_embeddings.npy


#2단계: 텍스트 패턴 vs 위험요인 가설 검정 코드

In [12]:
!pip install -q scikit-learn pingouin

from sklearn.decomposition import PCA
from scipy.stats import mannwhitneyu, kruskal, spearmanr
import pingouin as pg
import numpy as np
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 7.4 MB/s eta 0:00:00


In [13]:
#SBERT 임베딩 불러오기
emb = np.load("/content/drive/MyDrive/text_embeddings.npy")
print(emb.shape)   # (N, 768)

(1059, 768)


In [14]:
#PCA로 텍스트 패턴 요약(가장 중요한 축 1개 추출)
pca = PCA(n_components=1)
df["text_pc1"] = pca.fit_transform(emb)
print(df["text_pc1"].head())

0   -4.309988
1    2.856891
2   -3.808181
3   -5.520351
4    2.048222
Name: text_pc1, dtype: float32


/tmp/ipython-input-1141293661.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["text_pc1"] = pca.fit_transform(emb)


##검정 1 — 가격대 vs 텍스트 패턴 (Mann–Whitney U test)
목표

싼 매물 vs 나머지 매물 사이에서
text_pc1이 통계적으로 다르면
→ 텍스트가 위험요인(저가 유혹형 매물)과 연관됨 → 가설2 지지

In [15]:
#평당 가격 생성
df["price_per_area"] = df["price"] / df["exclusive_area"]

/tmp/ipython-input-218404983.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["price_per_area"] = df["price"] / df["exclusive_area"]


In [16]:
#저가 그룹(threshold: 하위 20%) 분리
thr = df["price_per_area"].quantile(0.2)

cheap = df[df["price_per_area"] <= thr]["text_pc1"]
normal = df[df["price_per_area"] > thr]["text_pc1"]

cheap.shape, normal.shape

((211,), (840,))

In [17]:
#Mann–Whitney U 검정 (비모수 2표본 검정)
stat, p = mannwhitneyu(cheap, normal, alternative="two-sided")

print("🔍 [검정1] 가격대 vs 텍스트 패턴")
print(" U-stat =", stat)
print(" p-value =", p)

🔍 [검정1] 가격대 vs 텍스트 패턴
 U-stat = 80621.5
 p-value = 0.04245704941147123


**유의함**

##검정 2 — 건물유형(building_usage) vs 텍스트 패턴 (Kruskal–Wallis test)
✔ 목표

건물유형별로 텍스트 패턴이 다르면
→ 유형 자체가 위험요인이고
→ 가설2 지지

In [18]:
#그룹별 데이터 준비
groups = df.groupby("building_usage")["text_pc1"].apply(list)
groups

,text_pc1
building_usage,
공동주택,"[-5.520351409912109, 2.4948577880859375, -5.55..."
교육연구시설,"[-2.910149335861206, -5.087638854980469]"
단독주택,"[-4.309987545013428, 2.856890916824341, -3.808..."
숙박시설,"[3.432445526123047, -6.113204479217529, 4.7838..."
업무시설,"[3.2926552295684814, 4.00216007232666, -4.7494..."
제1종 근린생활시설,"[2.1780903339385986, 1.6936912536621094, -6.67..."
제2종 근린생활시설,"[2.048222303390503, 0.35627156496047974, -1.90..."


In [19]:
#Kruskal–Wallis 검정
stat, p = kruskal(*groups)

print("🔍 [검정2] 건물 유형 vs 텍스트 패턴")
print(" H-stat =", stat)
print(" p-value =", p)

🔍 [검정2] 건물 유형 vs 텍스트 패턴
 H-stat = 18.78694315622384
 p-value = 0.004539000656516803


**유의함**

##검정 3 — 이미지 이상치 vs 텍스트 패턴 (Spearman 상관분석)

In [20]:
#Spearman rank correlation
corr, p = spearmanr(df["image_anomaly_score"], df["text_pc1"])

print("🔍 [검정3] 이미지 이상치 vs 텍스트 패턴")
print(" Spearman rho =", corr)
print(" p-value =", p)

🔍 [검정3] 이미지 이상치 vs 텍스트 패턴
 Spearman rho = 0.027109819822487464
 p-value = 0.3781362916278679


**유의하지 않음**

3개 중 2개가 유의함으로 유의한 가설